In [1]:
import pandas as pd
import ast
import copy

In [4]:
cta_statistics = pd.read_csv('../TableStatistics/cta_statistics.csv')
cta_statistics = cta_statistics.loc[cta_statistics['column_count'] >= 50]
cta_statistics

,cta_label,table_count,column_count,tables_and_densities
0,URL,258896,426186,{'Product_corememoriesco.com_September2020.jso...
1,Product/name,807306,807306,{'Product_corememoriesco.com_September2020.jso...
2,currency,778523,779395,{'Product_corememoriesco.com_September2020.jso...
3,price,728024,728024,{'Product_corememoriesco.com_September2020.jso...
4,Text,854319,991881,{'Product_corememoriesco.com_September2020.jso...
...,...,...,...,...
96,JobPosting/name,152,152,{'JobPosting_electronicsforu.com_September2020...
97,VideoObject,49,50,{'Recipe_chefincamicia.com_September2020.json....
98,CreativeWork,128,128,{'Recipe_morocco-gold.com_September2020.json.g...
99,PublicationEvent,91,91,{'Book_montellolibrary.org_September2020.json....


In [6]:
#Choose tables with more than 3 columns
tabs = pd.read_csv('../expanded_tables_annotations_cta.csv')
tabs = tabs.loc[(tabs['column_count'] >= 3)]
tabs

,class,file_name,column_count,number_of_rows,overall_table_density,all_cols,rel_labels,type_labels
0,Product,Product_corememoriesco.com_September2020.json.gz,6,72,100,"{'url': 100, 'name': 100, 'offers:pricecurrenc...","{'url': 'url', 'name': 'name', 'offers:pricecu...","{'url': 'URL', 'name': 'Museum/name', 'offers:..."
1,Event,Event_davidlohmueller.com_September2020.json.gz,7,20,88,"{'startdate': 100, 'name': 100, 'description':...","{'startdate': 'startDate', 'name': 'name', 'de...","{'startdate': 'DateTime', 'name': 'Museum/name..."
2,Product,Product_winehousegvl.com_September2020.json.gz,5,43,100,"{'name': 100, 'sku': 100, 'offers:pricecurrenc...","{'name': 'name', 'sku': 'sku', 'offers:pricecu...","{'name': 'Museum/name', 'sku': 'IdentifierAT',..."
3,Product,Product_idaw.eu_September2020.json.gz,6,183,94,"{'description': 74, 'name': 100, 'image': 100,...","{'description': 'description', 'name': 'name',...","{'description': 'Text', 'name': 'Museum/name',..."
4,Product,Product_juvenileplanet.com_September2020.json.gz,5,10,88,"{'name': 100, 'sku': 80, 'offers:price': 80, '...","{'name': 'name', 'sku': 'sku', 'offers:price':...","{'name': 'Museum/name', 'sku': 'IdentifierAT',..."
...,...,...,...,...,...,...,...,...
1030076,Product,Product_wolfchild.com_September2020.json.gz,6,30,100,"{'name': 100, 'description': 100, 'image': 100...","{'name': 'name', 'description': 'description',...","{'name': 'Museum/name', 'description': 'Text',..."
1030077,Product,Product_aransweatersdirect.com_September2020.j...,4,84,100,"{'name': 100, 'description': 100, 'offers:pric...","{'name': 'name', 'description': 'description',...","{'name': 'Museum/name', 'description': 'Text',..."
1030078,Product,Product_mandystanley.com_September2020.json.gz,7,25,100,"{'name': 100, 'description': 100, 'offers:pric...","{'name': 'name', 'description': 'description',...","{'name': 'Museum/name', 'description': 'Text',..."
1030079,Product,Product_blksport.com_September2020.json.gz,8,338,98,"{'name': 100, 'description': 92, 'brand': 100,...","{'name': 'name', 'description': 'description',...","{'name': 'Museum/name', 'description': 'Text',..."


In [7]:
#CTA labels: column name to its CTA label (exclude None or Wrong annotations)
type_lbls = {}
for index, row in tabs.iterrows():
    for col in eval(row['type_labels']):
        if eval(row['type_labels'])[col] != None and eval(row['type_labels'])[col] != 'Wrong':
            type_lbls[col+'_'+row['file_name']] = eval(row['type_labels'])[col].strip()

In [9]:
file_names = set(tabs['file_name'].tolist())
len(file_names)

1010497

In [10]:
#Columns to ignore in selection with a density less than 70%
ignore = []
tables_to_dict = tabs.to_dict('records')
for table in tables_to_dict:
    for col in eval(table['all_cols']):
        if eval(table['all_cols'])[col] < 70:
            ignore.append(col+'_'+table['file_name'])
ignore = set(ignore)

In [26]:
#Create a file to hold already selected columns from tables and their selection type
# Selection type: 'Missing values', 'Intra similarity', 'Inter simimilarity/dissimilarity', 'format heterogeneity', 'rare column'

## 1. Corner Cases Columns Selection

In [12]:
num_sim = pd.read_csv('output-data/Similarities/subset8000_num_sim.csv.gz', compression='gzip')
num_sim2 = pd.read_csv('output-data/Similarities/subset6000_num_sim.csv.gz', compression='gzip')
num_sim3 = pd.read_csv('output-data/Similarities/subset6000_2_num_sim.csv.gz', compression='gzip')
date_sim = pd.read_csv('output-data/Similarities/subset6000_datetime_sim.csv.gz', compression='gzip')
date_sim2 = pd.read_csv('output-data/Similarities/subset7000_datetime_sim.csv.gz', compression='gzip')
date_sim3 = pd.read_csv('output-data/Similarities/subset7000_2_datetime_sim.csv.gz', compression='gzip')
text_sim = pd.read_csv('output-data/Similarities/subset260000_textcols_coltype_sim.csv.gz', compression='gzip')
text_sim2 = pd.read_csv('output-data/Similarities/subset170000_textcols_coltype_sim.csv.gz', compression='gzip')
similarities = pd.concat([num_sim, num_sim2, num_sim3, date_sim, date_sim2, date_sim3, text_sim, text_sim2], ignore_index=True)
similarities['file_name'] = similarities['col_name'].apply(lambda row: row.split('_')[1]+'_'+row.split('_')[2]+'_'+row.split('_')[3])
similarities = similarities.loc[similarities['file_name'].isin(file_names)]
similarities = similarities.loc[~similarities['col_name'].isin(ignore)]
similarities

,col_name,similar_cols,Unnamed: 0,file_name
0,offers:price_Product_winehousegvl.com_Septembe...,aggregaterating:reviewcount_Event_southcarolin...,NaN,Product_winehousegvl.com_September2020.json.gz
1,offers:price_Product_juvenileplanet.com_Septem...,nutrition:carbohydratecontent_Recipe_findus.se...,NaN,Product_juvenileplanet.com_September2020.json.gz
2,offers:price_Product_uppolomallets.com_Septemb...,aggregaterating:reviewcount_LocalBusiness_prom...,NaN,Product_uppolomallets.com_September2020.json.gz
3,offers:price_Product_tradefireworks.co.uk_Sept...,aggregaterating:ratingcount_Hotel_toubiz.de_Se...,NaN,Product_tradefireworks.co.uk_September2020.jso...
4,offers:price_Product_andgifts.co.uk_September2...,manufacturer_Product_lespetitslivres.com_Septe...,NaN,Product_andgifts.co.uk_September2020.json.gz
...,...,...,...,...
203796,name_MusicAlbum_elviscarden.com_September2020....,track:name_MusicAlbum_elviscarden.com_Septembe...,170245.0,MusicAlbum_elviscarden.com_September2020.json.gz
203797,track:name_MusicAlbum_elviscarden.com_Septembe...,name_MusicAlbum_elviscarden.com_September2020....,170248.0,MusicAlbum_elviscarden.com_September2020.json.gz
203798,educationrequirements_JobPosting_plantae.org_S...,partofseries_TVEpisode_freeonlinemovieshd.com_...,170249.0,JobPosting_plantae.org_September2020.json.gz
203799,address_LocalBusiness_ml.com_September2020.jso...,address:streetaddress_Person_lawlink.com_Septe...,170251.0,LocalBusiness_ml.com_September2020.json.gz


### Select intra similarity columns

In [13]:
#Column -> its similar cols
intra_similarities = {}
for index, row in similarities.iterrows():
    col_name, class_name, table_name, ending = row['col_name'].split('_')
    file_name = '_'.join([class_name, table_name, ending])
    
    #If correct column
    if row['col_name'] in type_lbls and (type_lbls[row['col_name']] != None or type_lbls[row['col_name']] != 'Wrong'):
        
        #Look at only its 10 most similar columns
        similar_cols = row['similar_cols'].split('; ')[:10]
        intra_similarities[row['col_name']] = []

        for col in similar_cols:
            col_name2, class_name2, table_name2, ending2 = col.split('_')
            file_name_2 = '_'.join([class_name2, table_name2, ending2])

            # More than 70% of density, same table and in selected tables for CPA
            if col in type_lbls and file_name_2 in file_names and file_name == file_name_2 and col not in ignore:
                if type_lbls[row['col_name']] != type_lbls[col]:
                    if (type_lbls[col] != 'None' or type_lbls[col] != 'Wrong'):
                        intra_similarities[row['col_name']].append(col)

In [14]:
intra_similarities

{'offers:price_Product_winehousegvl.com_September2020.json.gz': [],
 'offers:price_Product_juvenileplanet.com_September2020.json.gz': [],
 'offers:price_Product_uppolomallets.com_September2020.json.gz': [],
 'offers:price_Product_tradefireworks.co.uk_September2020.json.gz': [],
 'offers:price_Product_andgifts.co.uk_September2020.json.gz': [],
 'offers:price_Product_ablissfullybeautifulboutique.com_September2020.json.gz': [],
 'offers:price_Product_provita.be_September2020.json.gz': [],
 'offers:price_Product_reddevildelivery.com_September2020.json.gz': [],
 'sku_Product_passionfury.com_September2020.json.gz': [],
 'offers:price_Product_grapevinyl.net_September2020.json.gz': [],
 'sku_Product_grapevinyl.net_September2020.json.gz': [],
 'sku_Product_offonlineshop.com_September2020.json.gz': [],
 'offers:price_Product_offonlineshop.com_September2020.json.gz': [],
 'offers:price_Product_vanderzee.ca_September2020.json.gz': [],
 'offers:price_Product_katwalla.com_September2020.json.gz': [],

In [17]:
intra_tables = {} # Mark which columns can be selected in each table
intra_cols = set() # Which columns selected overall
intra_class = {} # How many columns per schema.org type/class

for key in intra_similarities:
    if intra_similarities[key]:
        
        table = key.split('_')[1] + '_' + key.split('_')[2] + '_' + key.split('_')[3]
        class_ = key.split('_')[0]
        
        if table not in intra_tables:
            intra_tables[table] = set()
        
        if class_ not in intra_class:
            intra_class[class_] = 0
            
        intra_cols.add(key)
        intra_tables[table].add(key)
        intra_class[class_] += 1
        
        for col in intra_similarities[key]:
            intra_cols.add(col)
            intra_tables[table].add(col)
            intra_class[class_] += 1
            

In [24]:
#Number of Selected tables
len(intra_tables)

6928

In [25]:
#Number of Selected cols
len(set(intra_cols))

15271

In [27]:
#Mark all selected columns in a dictionary:
selected_cols_tables = {}

for table in intra_tables:
    
    selected_cols_tables[table] = {}
    for col in intra_tables[table]:
        selected_cols_tables[table][col.split('_')[0]] = 'Intra similarity'

In [22]:
low_class = [] #Schema.org types that have less than 1500 selected columns until now
low_label = [] #CTA labels that have less than 100 examples until now

for cl in intra_class:
    if intra_class[cl] < 1500:
        low_class.append(cl)
        
for lb in intra_labels:
    if len(intra_labels[lb]) < 100:
        low_label.append(lb)

### Select inter table similarity columns

In [29]:
inter_similarities = {}
for index, row in similarities.iterrows():
    col_name, class_name, table_name, ending = row['col_name'].split('_')
    file_name = '_'.join([class_name, table_name, ending])
        
    #If correct column
    if row['col_name'] in type_lbls and type_lbls[row['col_name']] != None and type_lbls[row['col_name']] != 'Wrong':
        #Look at 10 most similar columns
        similar_cols = row['similar_cols'].split('; ')[:10]
        inter_similarities[row['col_name']] = []

        for col in similar_cols:
            col_name2, class_name2, table_name2, ending2 = col.split('_')
            file_name_2 = '_'.join([class_name2, table_name2, ending2])

            if col in type_lbls and file_name_2 in file_names and file_name != file_name_2 and col not in ignore:
                if type_lbls[row['col_name']] != type_lbls[col]:
                    if (type_lbls[col] != None and type_lbls[col] != 'Wrong'):
                        inter_similarities[row['col_name']].append(col)

In [30]:
inter_similarities

{'offers:price_Product_winehousegvl.com_September2020.json.gz': ['offers:sku_Product_syberia.net_September2020.json.gz'],
 'offers:price_Product_juvenileplanet.com_September2020.json.gz': ['nutrition:carbohydratecontent_Recipe_moderncook.com.vn_September2020.json.gz',
  'aggregaterating:reviewcount_Restaurant_food94.com_September2020.json.gz',
  'aggregaterating:reviewcount_Hotel_toptours.hu_September2020.json.gz',
  'geo:latitude_Museum_trekraw.com_September2020.json.gz'],
 'offers:price_Product_uppolomallets.com_September2020.json.gz': ['aggregaterating:reviewcount_LocalBusiness_promobil.de_September2020.json.gz',
  'nutrition:cholesterolcontent_Recipe_faylinameir.com_September2020.json.gz',
  'offers:mpn_Product_cyclony.ru_September2020.json.gz',
  'category_Product_calcuttboatsshop.com_September2020.json.gz',
  'nutrition:cholesterolcontent_Recipe_diabeticdinnerrecipes.com_September2020.json.gz',
  'nutrition:carbohydratecontent_Recipe_faylinameir.com_September2020.json.gz'],
 'off

In [31]:
#Select columns that have at least one similar column
inter_sims = [ [x] + inter_similarities[x] for x in inter_similarities if len(inter_similarities[x]) > 0]

In [48]:
# Selecting inter similarities: 
#Count how many columns for low class number of columns and for low CTA labels can be selected
select_test = []
i = 0

for cols in inter_sims:
    select_test.append([])
    
    for col in cols:
        class_ = col.split('_')[1]
        
        #If table is in low class and in low label
        if class_ in low_class and type_lbls[col] in low_label:
            #If it hasn't been selected already in the intra similarity phase
            if (table in selected_cols_tables and col.split('_')[0] not in selected_cols_tables[table]) or table not in selected_cols_tables:
                select_test[i].append(col)
    i+=1
    
    
# And Filter out columns with less than 1 similar column
s = [ x for x in select_test if len(x) > 1]


# And Select maximum 3500 cols for each CTA label
sel_labels = {}

for sim in s:
    for col in sim:
        
        if type_lbls[col] not in sel_labels:
            sel_labels[type_lbls[col]] = []
        
        if len(sel_labels[type_lbls[col]]) < 3500:
            sel_labels[type_lbls[col]].append(col)

sel_labels

{'price': ['offers:price_Product_juvenileplanet.com_September2020.json.gz',
  'offers:price_Product_uppolomallets.com_September2020.json.gz',
  'offers:price_Product_tradefireworks.co.uk_September2020.json.gz',
  'offers:price_Product_andgifts.co.uk_September2020.json.gz',
  'offers:price_Product_ablissfullybeautifulboutique.com_September2020.json.gz',
  'offers:price_Product_provita.be_September2020.json.gz',
  'offers:price_Product_reddevildelivery.com_September2020.json.gz',
  'offers:price_Product_grapevinyl.net_September2020.json.gz',
  'offers:price_Product_vanderzee.ca_September2020.json.gz',
  'offers:price_Product_katwalla.com_September2020.json.gz',
  'offers:price_Product_fabion.nl_September2020.json.gz',
  'offers:price_Product_praha-elektromaterial.cz_September2020.json.gz',
  'offers:price_Product_haytoncreeklavender.com_September2020.json.gz',
  'offers:price_Product_wallmounted-electricfires.co.uk_September2020.json.gz',
  'offers:price_Product_pineorchardwine.net_Septe

In [49]:
#Select columns which are included in sel_labels
selected_test_2 = []
i = 0

for cols in inter_sims:
    selected_test_2.append([])
    
    for col in cols:
        if type_lbls[col] in sel_labels and col in sel_labels[type_lbls[col]]:
            selected_test_2[i].append(col)
        
    i+=1

#Filter out columns with no similar columns
s_2 = [ x for x in selected_test_2 if len(x) > 1]

In [55]:
# Add the new selected columns to the already selected ones
selected_cols = copy.deepcopy(selected_cols_tables)
for s in s_2:
    for col in s:
        tab = col.split('_')[1] + '_' + col.split('_')[2] + '_' + col.split('_')[3]
        
        if tab not in selected_cols:
            selected_cols[tab] = {}
        
        selected_cols[tab][col.split('_')[0]] = 'Inter Similarity'
        

## 2. Select Missing values columns

In [60]:
#Choose some low density tables
annotated_tables = pd.read_csv('output-data/statistics/expanded_tables_annotations_corrected.csv')
low_dens_70 = annotated_tables.loc[ (annotated_tables['overall_table_density'] < 70) ]
low_dens_70

,class,file_name,column_count,number_of_rows,overall_table_density,all_cols,rel_labels,type_labels
14,LocalBusiness,LocalBusiness_litmind.com_September2020.json.gz,9,63,47,"{'telephone': 87, 'name': 100, 'address:addres...","{'telephone': 'telephone', 'name': 'name', 'ad...","{'telephone': 'telephone', 'name': 'Museum/nam..."
41,Person,Person_altervista.org_September2020.json.gz,2,624,53,"{'name': 100, 'description': 7}","{'name': 'name', 'description': 'description'}","{'name': 'Museum/name', 'description': 'Text'}"
54,Product,Product_internetprodavnice.rs_September2020.js...,10,39,63,"{'description': 74, 'name': 100, 'url': 74, 'o...","{'description': 'description', 'name': 'name',...","{'description': 'Text', 'name': 'Museum/name',..."
267,Product,Product_tissusprovencaux.fr_September2020.json.gz,13,17,39,"{'name': 100, 'description': 23, 'sku': 5, 'br...","{'name': 'name', 'description': 'description',...","{'name': 'Museum/name', 'description': 'Text',..."
271,Product,Product_budomaster.ru_September2020.json.gz,10,40,62,"{'description': 100, 'name': 100, 'sku': 100, ...","{'description': 'description', 'name': 'name',...","{'description': 'Text', 'name': 'Museum/name',..."
...,...,...,...,...,...,...,...,...
1045999,Product,Product_farmaciamarket.es_September2020.json.gz,11,344,69,"{'name': 100, 'description': 34, 'url': 21, 'b...","{'name': 'name', 'description': 'description',...","{'name': 'Museum/name', 'description': 'Text',..."
1046052,Product,Product_wineo.de_September2020.json.gz,6,16,66,"{'description': 50, 'brand': 100, 'width': 50,...","{'description': 'description', 'brand': 'brand...","{'description': 'Text', 'brand': 'Brand', 'wid..."
1046129,Recipe,Recipe_herbncowgirl.com_September2020.json.gz,10,19,62,"{'name': 94, 'recipeingredient': 68, 'datepubl...","{'name': 'name', 'recipeingredient': 'recipeIn...","{'name': 'Museum/name', 'recipeingredient': 'T..."
1046194,Product,Product_sibpech.ru_September2020.json.gz,10,14,67,"{'name': 100, 'description': 7, 'offers:pricec...","{'name': 'name', 'description': 'description',...","{'name': 'Museum/name', 'description': 'Text',..."


In [61]:
#How many low density columns can be annotated per table
low_dens_70['low_cols'] = low_dens_70['all_cols'].apply(lambda row: len( [x for x in eval(row) if eval(row)[x] < 70 and eval(row)[x] > 10 ] ))

/tmp/ipykernel_2430765/755326462.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  low_dens_70['low_cols'] = low_dens_70['all_cols'].apply(lambda row: len( [x for x in eval(row) if eval(row)[x] < 70 and eval(row)[x] > 10 ] ))


In [64]:
low_dens_70.loc[ (low_dens_70['low_cols'] >= 3) ].groupby(['class'])['low_cols'].sum()

class
Book                643
CreativeWork       1278
Event              7437
Hotel               664
JobPosting          256
LocalBusiness      2041
Movie              1875
MusicAlbum           17
MusicRecording       35
Person             1391
Place               562
Product           37881
Recipe            11077
Restaurant          337
SportsEvent          54
TVEpisode            36
Name: low_cols, dtype: int64

In [66]:
# Limit Product, Recipe and Event tables to 800
prods = low_dens_70.loc[ (low_dens_70['low_cols'] >= 3) & (low_dens_70['class'] == 'Product' ) ][800:]
recipe = low_dens_70.loc[ (low_dens_70['low_cols'] >= 3) & (low_dens_70['class'] == 'Recipe' ) ][800:]
event = low_dens_70.loc[ (low_dens_70['low_cols'] >= 3) & (low_dens_70['class'] == 'Event' ) ][800:]

remove_tables = prods['file_name'].tolist() + recipe['file_name'].tolist() + event['file_name'].tolist()

In [69]:
#Select columns with low density from the low density tables that have at least 3 low density columns
#Add to all other selected columns

for index, row in low_dens_70.loc[ (low_dens_70['low_cols'] >= 3) ].iterrows():
    file_name = row['file_name']
    
    if file_name not in remove_tables:    
        if file_name not in selected_cols:
            selected_cols[file_name] = {}

        #Look at low density columns and select if not already annotated from corner cases
        annotated_cols = eval(row['all_cols'])

        for column in annotated_cols:
            if annotated_cols[column] < 70 and annotated_cols[column] > 10 and column not in selected_cols[file_name]:
                selected_cols[file_name][column] = 'Missing values'


## 3. Select value format heterogeneity columns

In [ ]:
#Select the same columns that were selected for CPA excpet from some datetime columns
selection_cpa = pd.read_csv('output-data/cpa-datasets/selected_1_2.csv')
date_f = open("../datetime_select.txt", 'r')
date = [line.replace('\n', '') for line in date_f.readlines()]

In [ ]:
selection = annotated_tables.loc[ annotated_tables['file_name'].isin(selected_cols) ]
selection['selected_cols'] = selected_cols

In [ ]:
for index, row in selection_cpa.iterrows():
    types = eval(row['type_labels'])
    for col in eval(row['selected_cols']):
        if col != 'basesalary' and col != 'estimatedsalary' and col != 'basesalary' and col != 'nutrition:servingsize' and col != 'size':
            if eval(row['selected_cols'])[col] == 'Value Heterogeneity':
                if (row['file_name'] in selected_cols and col not in selected_cols[row['file_name']]) or row['file_name'] not in selected_cols:
                    if (types[col] == 'DateTime' or types[col] == 'Date'):
                        if col+'_'+row['file_name'] in date:
                            if row['file_name'] not in selected_cols:
                                selected_cols[row['file_name']] = {}
                            selected_cols[row['file_name']][col] = 'Value Heterogeneity'
                        
                    else:
                    
                        if row['file_name'] not in selected_cols:
                            selected_cols[row['file_name']] = {}
                        selected_cols[row['file_name']][col] = 'Value Heterogeneity'

## 4. Select random columns for all labels

In [ ]:
all_cta = cta_statistics['cta_label'].tolist()

In [ ]:
type_lbls = {}
for index, row in annotated_tables.iterrows():
    for col in eval(row['type_labels']):
        if eval(row['type_labels'])[col] != None and eval(row['type_labels'])[col] != 'Wrong':
            type_lbls[col+'_'+row['file_name']] = eval(row['type_labels'])[col].strip() 

In [ ]:
selection = annotated_tables.loc[ annotated_tables['file_name'].isin(selected_cols) ]
selection['selected_cols'] = selected_cols
selection['selected_cols_number'] = selection['selected_cols'].apply(lambda row: len(eval(row)))

In [ ]:
selection_dict = selection.to_dict('records')

In [ ]:
#Select from already selected tables all non-annotated columns with a limit of 6500 columns
count_class = {} #Count columns per Schema.org type/class
type_count = {} #Count columns per CTA label

for tab in selection_dict:
    sel = eval(tab['selected_cols'])
    
    if tab['class'] not in count_class:
        count_class[tab['class']] = 0
    
    #Select a maximum of 6500 columns per Schema.org type/class 
    if count_class[tab['class']] < 6500:
    
        for col in eval(tab['all_cols']):
            #If column has not been yet selected:
            if col not in sel:
                count_class[tab['class']] += 1
                #Add to selected
                selected_cols[tab['file_name']][col] = 'Random'

                if type_lbls[col+'_'+tab['file_name']] not in type_count:
                    type_count[type_lbls[col+'_'+tab['file_name']]] = 0

                type_count[type_lbls[col+'_'+tab['file_name']]] += 1


            else:
                if type_lbls[col+'_'+tab['file_name']] not in type_count:
                    type_count[type_lbls[col+'_'+tab['file_name']]] = 0

                type_count[type_lbls[col+'_'+tab['file_name']]] += 1

In [ ]:
all_cta = all_cta + list(type_count.keys())
all_cta = list(set(all_cta))
#Mark only labels that have not yet reached at least 100 examples
for c in all_cta:
    if c in type_count and type_count[c] >= 100: 
        all_cta.remove(c)

In [ ]:
#Add more examples to CTA labels that have not reached enough examples
count_class_2 = {} #Count columns per Schema.org type, do not pass 6500 columns

for cl in count_class:
    count_class_2[cl] = 0
    if count_class[cl] < 2000:
        d = annotated_tables.loc[(~annotated_tables['file_name'].isin(selected_cols)) & (annotated_tables['class'] == cl ) & (annotated_tables['overall_table_density'] >= 70 )]
        
        for index, row in d.iterrows():
            for col in eval(row['all_cols']):
                if type_count[type_lbls[col+'_'+row['file_name']]] < 2000:
                    if count_class_2[cl] < (6500 - count_class[cl]):
                        if row['file_name'] not in selected_cols:
                            selected_cols[row['file_name']] = {}
                        selected_cols[row['file_name']][col] = 'Random 2'
                        count_class_2[cl] += 1
        

In [ ]:
#Select maximum 300 columns per CTA label
new_cols = {}
for l in all_cta:
    new_cols[l] = []

for index, row in annotated_tables.sort_values('column_count', ascending=False).iterrows():
    types = eval(row['type_labels'])
    
    for col in eval(row['all_cols']):
        if types[col] in all_cta and len(new_cols[types[col]]) < 300:
                
            if row['file_name'] not in selected_cols or col not in selected_cols[row['file_name']]:
                
                new_cols[types[col]].append(col+'_'+row['file_name'])
                
                #add to selected columns
                if row['file_name'] not in selected_cols:
                    selected_cols[row['file_name']] = {}
                selected_cols[row['file_name']][col] = 'Random 3'
                

## Assemble all selected columns

In [75]:
selection = annotated_tables.loc[ annotated_tables['file_name'].isin(selected_cols) ]
sel_cols = []
for index, row in selection.iterrows():
    sel_cols.append(selected_cols[row['file_name']])
selection['selected_cols'] = sel_cols
selection

,class,file_name,column_count,number_of_rows,overall_table_density,all_cols,rel_labels,type_labels,selected_cols
0,Product,Product_corememoriesco.com_September2020.json.gz,6,72,100,"{'url': 100, 'name': 100, 'offers:pricecurrenc...","{'url': 'url', 'name': 'name', 'offers:pricecu...","{'url': 'URL', 'name': 'Museum/name', 'offers:...",{'offers:pricecurrency': 'Inter Similarity'}
1,Event,Event_davidlohmueller.com_September2020.json.gz,7,20,88,"{'startdate': 100, 'name': 100, 'description':...","{'startdate': 'startDate', 'name': 'name', 'de...","{'startdate': 'DateTime', 'name': 'Museum/name...","{'startdate': 'Inter Similarity', 'enddate': '..."
2,Product,Product_winehousegvl.com_September2020.json.gz,5,43,100,"{'name': 100, 'sku': 100, 'offers:pricecurrenc...","{'name': 'name', 'sku': 'sku', 'offers:pricecu...","{'name': 'Museum/name', 'sku': 'IdentifierAT',...",{'offers:pricecurrency': 'Inter Similarity'}
3,Product,Product_idaw.eu_September2020.json.gz,6,183,94,"{'description': 74, 'name': 100, 'image': 100,...","{'description': 'description', 'name': 'name',...","{'description': 'Text', 'name': 'Museum/name',...","{'name': 'Intra similarity', 'description': 'I..."
4,Product,Product_juvenileplanet.com_September2020.json.gz,5,10,88,"{'name': 100, 'sku': 80, 'offers:price': 80, '...","{'name': 'name', 'sku': 'sku', 'offers:price':...","{'name': 'Museum/name', 'sku': 'IdentifierAT',...","{'offers:price': 'Inter Similarity', 'offers:p..."
...,...,...,...,...,...,...,...,...,...
1045816,Person,Person_andersonfuneralhomegcwi.com_September20...,6,19,98,"{'description': 100, 'givenname': 100, 'birthd...","{'description': 'description', 'givenname': 'g...","{'description': 'Text', 'givenname': 'Text', '...",{'deathdate': 'Inter Similarity'}
1045895,Book,Book_stital.ac.id_September2020.json.gz,9,18,100,"{'name': 100, 'author': 100, 'bookformat': 100...","{'name': 'name', 'author': 'author', 'bookform...","{'name': 'Museum/name', 'author': 'Person', 'b...",{'inlanguage': 'Inter Similarity'}
1046083,LocalBusiness,LocalBusiness_suffolkcountyhousebuyers.com_Sep...,11,10,94,"{'name': 100, 'telephone': 90, 'address:addres...","{'name': 'name', 'telephone': 'telephone', 'ad...","{'name': 'Museum/name', 'telephone': 'telephon...",{'openinghours': 'Inter Similarity'}
1046156,MusicAlbum,MusicAlbum_elviscarden.com_September2020.json.gz,5,21,99,"{'name': 100, 'numtracks': 100, 'track:inalbum...","{'name': 'name', 'numtracks': 'numTracks', 'tr...","{'name': 'Museum/name', 'numtracks': 'Integer'...","{'name': 'Intra similarity', 'track:name': 'In..."


In [ ]:
res = []
for index, row in selection.iterrows():
    rel_labels = eval(row['rel_labels'])
    type_labels = eval(row['type_labels'])
    densities = eval(row['all_cols'])
    
    for cols in selected_cols_4[row['file_name']]:
        if cols in rel_labels and cols in type_labels:
            res.append([row['class'], cols, row['file_name'], rel_labels[cols], type_lbls[cols+'_'+row['file_name']], densities[cols], selected_cols_4[row['file_name']][cols] ])
        

In [ ]:
dataset = pd.DataFrame(res, columns=['class', 'column_name', 'file_name', 'relation_label', 'type_label', 'density', 'selection_type'])
dataset

In [ ]:
#Remove columns belonging to CTA labels that do not have a minimum of 50 examples
s = dataset.groupby(['type_label'])['column_name'].count()
no_rels = list(s[s < 50].keys())
dataset = dataset.loc[~dataset['type_label'].isin(no_rels)]

In [ ]:
dataset.to_csv('output-data/cta-datasets/dataset_cta.csv', index=False)